# Food Delivery Data Analysis

This notebook analyzes food delivery data from three sources: CSV (orders), JSON (users), and SQL (restaurants).
It merges these datasets and answers key business questions.

In [2]:
import pandas as pd
import re
import ast

# Load CSV Data (Orders)
orders_df = pd.read_csv('orders.csv')
print("Orders Loaded:", orders_df.shape)

# Load JSON Data (Users)
users_df = pd.read_json('users.json')
print("Users Loaded:", users_df.shape)

# Load SQL Data (Restaurants)
# Parsing INSERT statements from the SQL file
restaurants_data = []
with open('restaurants.sql', 'r') as f:
    for line in f:
        if line.strip().startswith('INSERT INTO restaurants VALUES'):
            match = re.search(r'\((.*)\);', line)
            if match:
                values_str = match.group(1)
                try:
                    # Safely evaluate the tuple string
                    values = ast.literal_eval(f"({values_str})")
                    restaurants_data.append(values)
                except Exception as e:
                    print(f"Error parsing line: {e}")

restaurants_df = pd.DataFrame(restaurants_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])
print("Restaurants Loaded:", restaurants_df.shape)

Orders Loaded: (10000, 6)
Users Loaded: (3000, 4)
Restaurants Loaded: (500, 4)


## Data Merging
We perform a Left Join to combine orders with user and restaurant details.

In [3]:
# Merge Orders with Users
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')

# Merge with Restaurants
# We use suffixes to handle the duplicate 'restaurant_name' column found in orders.csv
final_df = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left', suffixes=('', '_master'))

# Convert order_date to datetime
final_df['order_date'] = pd.to_datetime(final_df['order_date'], format='%d-%m-%Y')

print("Final Dataset Shape:", final_df.shape)
final_df.head()

Final Dataset Shape: (10000, 12)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,restaurant_name_master,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


## Analysis
Answering key questions about revenue, user behavior, and restaurant performance.

In [4]:
# 1. Top Revenue Cities
city_revenue = final_df.groupby('city')['total_amount'].sum().sort_values(ascending=False)
print("Top Revenue Cities:\n", city_revenue)

# 2. Top Cuisines by Revenue
cuisine_revenue = final_df.groupby('cuisine')['total_amount'].sum().sort_values(ascending=False)
print("\nTop Cuisines:\n", cuisine_revenue)

# 3. Average Order Value by Membership
membership_aov = final_df.groupby('membership')['total_amount'].mean()
print("\nAverage Order Value by Membership:\n", membership_aov)

Top Revenue Cities:
 city
Bangalore    2206946.58
Chennai      1990513.03
Pune         1924797.93
Hyderabad    1889366.58
Name: total_amount, dtype: float64

Top Cuisines:
 cuisine
Mexican    2085503.09
Italian    2024203.80
Indian     1971412.58
Chinese    1930504.65
Name: total_amount, dtype: float64

Average Order Value by Membership:
 membership
Gold       797.145556
Regular    805.158434
Name: total_amount, dtype: float64


In [5]:
# Save the final dataset
final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("Dataset saved successfully!")

Dataset saved successfully!
